### Install dependencies and load packages

In [ ]:
!pip3 install torchmetrics==0.4.1
!pip3 install transformers==4.8.2
!pip3 install pytorch_lightning==1.3.8

In [10]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
from torch.utils.data import Dataset
from MLN_individual_files.helper_classes import *
import numpy as np
import pickle
from transformers import get_scheduler
from transformers import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

## Preview data and fine-tune MLN model.

In [3]:
model = T5ForConditionalGeneration.from_pretrained('ufal/byt5-small-multilexnorm2021-da')
tokenizer = AutoTokenizer.from_pretrained('ufal/byt5-small-multilexnorm2021-da')

Downloading: 100%|██████████| 706/706 [00:00<00:00, 227kB/s]
Downloading: 100%|██████████| 1.20G/1.20G [01:47<00:00, 11.1MB/s] 
Downloading: 100%|██████████| 2.59k/2.59k [00:00<00:00, 1.07MB/s]
Downloading: 100%|██████████| 2.50k/2.50k [00:00<00:00, 801kB/s]


In [5]:
with open('data/mln_data_test_inputs.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open('data/mln_data_test_outputs.pkl', 'rb') as f:
    y_train = pickle.load(f)

data = MultiPlexDataset(X_train, y_train, only_include_corrections=True)

Dataset initialized...


In this demo the test set is loaded. In the real training we ofcourse used the training set. 
We have only included data points with errors.

In [7]:
data[0] #We see that there is an error.

{'input_sample': 'gyset der har siddet sammenkrøbet i <extra_id_0>nakke<extra_id_1> regionen udløses',
 'expected_output': 'nakkeregionen'}

In [11]:
dataloader = DataLoader(data, batch_size=8, collate_fn=CollateFunctor_Train(tokenizer))

In [12]:
optimizer = AdamW(model.parameters(), lr=0.3e-3)
num_epochs = 1
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=4000,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

2624


In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
running_loss=0
model.train()
for i, batch in enumerate(dataloader):
    batch = {k: v.to(device) for k, v in batch.items() if k != 'sentence_ids' and k != 'word_ids'}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)
    # print statistics
    running_loss += loss.item()
    if i % 50 == 0:    # print every 2000 mini-batches
        print('[%5d] loss: %.3f' %
              (i + 1, running_loss / 50))
        running_loss = 0.0

The MLN model was fine-tuned and the model was uploaded to huggingface.
https://huggingface.co/jenspt/mln_ft 